In [80]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM, Dropout
from tensorflow.keras.utils import to_categorical

In [81]:
# load text and covert to lowercase
upload = "wonderland_simplied.txt"
text = open(upload, 'r', encoding='latin-1').read()

text = text.lower() #vocabulary reduction

In [82]:
unique_chars = sorted(list(set(text)))
chars_to_ints = dict((c, i) for i, c in enumerate(unique_chars))
ints_to_chars = dict((i, c) for i, c in enumerate(unique_chars))

In [83]:
print(unique_chars)
print(chars_to_ints)
print(ints_to_chars)

['\n', ' ', '!', '(', ')', '*', ',', '-', '.', '0', '3', ':', ';', '?', '[', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x91', '\x92', '\x93', '\x94']
{'\n': 0, ' ': 1, '!': 2, '(': 3, ')': 4, '*': 5, ',': 6, '-': 7, '.': 8, '0': 9, '3': 10, ':': 11, ';': 12, '?': 13, '[': 14, '_': 15, 'a': 16, 'b': 17, 'c': 18, 'd': 19, 'e': 20, 'f': 21, 'g': 22, 'h': 23, 'i': 24, 'j': 25, 'k': 26, 'l': 27, 'm': 28, 'n': 29, 'o': 30, 'p': 31, 'q': 32, 'r': 33, 's': 34, 't': 35, 'u': 36, 'v': 37, 'w': 38, 'x': 39, 'y': 40, 'z': 41, '\x91': 42, '\x92': 43, '\x93': 44, '\x94': 45}
{0: '\n', 1: ' ', 2: '!', 3: '(', 4: ')', 5: '*', 6: ',', 7: '-', 8: '.', 9: '0', 10: '3', 11: ':', 12: ';', 13: '?', 14: '[', 15: '_', 16: 'a', 17: 'b', 18: 'c', 19: 'd', 20: 'e', 21: 'f', 22: 'g', 23: 'h', 24: 'i', 25: 'j', 26: 'k', 27: 'l', 28: 'm', 29: 'n', 30: 'o', 31: 'p', 32: 'q', 33: 'r', 34: 's', 35: 't', 36: 'u', 37: 'v', 38: 

In [84]:
import pandas as pd
text[:10] #learn the meaning of text

'*** start '

In [85]:
len(unique_chars)

46

In [86]:
len(text)

32182

In [87]:
seq_in = text[1:1 + 50]
print("seq_in:",seq_in)
seq_out = text[51]
print("seq_out:",seq_out)

seq_in: ** start of this project gutenberg ebook alices a
seq_out: d


In [88]:
dataX=[]
print(seq_in)
dataX.append([chars_to_ints[char] for char in seq_in])
print(dataX)

** start of this project gutenberg ebook alices a
[[5, 5, 1, 34, 35, 16, 33, 35, 1, 30, 21, 1, 35, 23, 24, 34, 1, 31, 33, 30, 25, 20, 18, 35, 1, 22, 36, 35, 20, 29, 17, 20, 33, 22, 1, 20, 17, 30, 30, 26, 1, 16, 27, 24, 18, 20, 43, 34, 1, 16]]


In [89]:
dataY = []
print(seq_out)
dataY.append(chars_to_ints[seq_out])
print(dataY)

d
[19]


In [90]:
seq_length = 100
dataX, dataY = [], []

In [91]:
for i in range(0, len(text) - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([chars_to_ints[char] for char in seq_in])
    dataY.append(chars_to_ints[seq_out])

In [92]:
n_patterns = len(dataX)
print(n_patterns)

32082


In [93]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
#X

In [94]:
# normalize
X = X / float(len(unique_chars))
#print(X)

In [95]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False, categories='auto')
y = encoder.fit_transform(np.array(dataY).reshape(-1, 1))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [96]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.8))

model.add(LSTM(256))
model.add(Dropout(0.8))

model.add(Dense(len(unique_chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [97]:
# Train the model (you can adjust epochs and batch_size)
model.fit(X, y, epochs=10, batch_size=2000, verbose=1)

Epoch 1/10
17/17 [==============================] - 232s 13s/step - loss: 3.4299
Epoch 2/10
17/17 [==============================] - 227s 13s/step - loss: 3.1902
Epoch 3/10
17/17 [==============================] - 228s 13s/step - loss: 3.1390
Epoch 4/10
17/17 [==============================] - 226s 13s/step - loss: 3.1156
Epoch 5/10
17/17 [==============================] - 226s 13s/step - loss: 3.0963
Epoch 6/10
17/17 [==============================] - 226s 13s/step - loss: 3.0838
Epoch 7/10
17/17 [==============================] - 226s 13s/step - loss: 3.0747
Epoch 8/10
17/17 [==============================] - 226s 13s/step - loss: 3.0720
Epoch 9/10
17/17 [==============================] - 226s 13s/step - loss: 3.0646
Epoch 10/10
17/17 [==============================] - 227s 13s/step - loss: 3.0602


In [100]:
#generating the text
# pick a random seed
random = np.random.randint(0, len(dataX)-1)
pattern = dataX[random]
print("\"", ''.join([ints_to_chars[value] for value in pattern]), "\"")


" ere me?

well, perhaps not, said alice in a soothing tone: dont be angry
about it. and yet i wi "


In [109]:
#generate based on the input text
def generate_text(seed_text, n_chars):
    seed_text=seed_text.lower()
    generated_text = seed_text
    pattern = [chars_to_ints[char] for char in seed_text]

    while len(pattern) < 100:
        pattern.insert(0, 0)

    output=seed_text
    for i in range(n_chars):
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(len(unique_chars))
        prediction = model.predict(x, verbose=0)
        index = np.argmax(prediction)
        result = ints_to_chars[index]
        output += result
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
    return output

In [110]:
print(generate_text("Alice was beginning",200)) # I don't know how to fix this problem, and I found from the comments there are a lot of people hving same question

alice was beginning                                                                                                                                                                                                        
